In [1]:
import math #basic python math library, used for square rooting and getting pi
import numpy as np #common library, source of numpy arrays which are easier to manipulate as vectors than your average python list

from scipy.constants import e, m_e, epsilon_0 
#physics constants I need, the charge of a proton or electron (e), the mass of an electron (m_e), and epsilon naught

import matplotlib.pyplot as plt #for plotting data
import mpl_toolkits.mplot3d.axes3d as p3 #allowing for 3d axis instead of 2d
import matplotlib.animation as animation #to create animation
from IPython.display import HTML #making it possible to display my animation

In [2]:
class Charge: #class to create Charge objects, which will make it easier to work with charges

    def __init__(self, x, y, z, charge): #initiation function, magic method
        self.position = np.array([float(x), float(y), float(z)]) #creating the position, using the given x, y, and z coordinates, as an array
        self.charge = charge #getting the charge of the particle

    def get_position(self): #function to get the position of a charge
        return self.position #return the position

    def get_charge(self): #function to get the charge of a charge
        return self.charge #return the charge

In [8]:
class Electron: #class to create Electron objects, which will allow me to make functions for electrons

    def __init__(self, x, y, z): #initiation function, magic method
        self.position = np.array([float(x), float(y), float(z)]) #creating the position, using the given x, y, and z coordinates, as an array
        self.charge = -e #all electrons have a charge of -e
        self.velocity = np.array([0.0, 0.0, 0.0]) #start with 0 velocity
        self.acceleration = np.array([0.0, 0.0, 0.0]) #start with 0 acceleration

    def get_position(self): #function to get the position of an electrno
        return self.position #return the position

    def get_force(self, charges): #function to get the sum of all the forces on an electron
        force = np.array([0.0, 0.0, 0.0]) #start with 0
        for charge in charges: #then go through each charge
            r = math.sqrt((charge.get_position() - self.get_position())[0] ** 2 + (charge.get_position() - self.get_position())[1] ** 2)
            #get the distance from electron to charge
            force += (self.get_position() - charge.get_position()) * charge.get_charge() * self.charge / 4 / math.pi / epsilon_0 / (r ** 3)
            #use Coloumb's Law to find the force a single charge places on the electron and add it to the current force
        self.acceleration = force / m_e #Newton's second law, f=ma, so a=f/m
   
    def get_acceleration(self): #return the acceleration
        return self.acceleration #return acceleration

    def get_velocity(self):
        return self.velocity

    def update_velocity(self): #changing velocity
        self.velocity += self.acceleration #small increments, velocity changes by acceleration

    def update_position(self): #changing position
        self.position += self.velocity #small increments, position changes by velocity

In [4]:
def electron_update(charges, electron): #function to update the state of the electron

    electron.get_force(charges) #getting the current force
    electron.update_velocity() #updating velocity
    electron.update_position() #updating position

In [5]:
i = 1 #start with one frame

def gen(charges, electron): #making a generator function!
    global i #letting us access i, prevents it from resetting
    sep = True #boolean to determine if you should continue
    while sep:
        print(i)
        for charge in charges: #look at each charge
            if abs(electron.get_position()[0] - charge.get_position()[0]) < 4 and abs(electron.get_position()[1] - charge.get_position()[1]) < 4 and abs(electron.get_position()[2] - charge.get_position()[2]) < 4:
            #if the electron is practically on the charge
                sep = False #don't continue
                print("on charge") #let us know we are on a charge
                print(charge.get_position(), electron.get_position()) #print both locations
                print(abs(electron.get_position()[0] - charge.get_position()[0]), abs(electron.get_position()[1] - charge.get_position()[1]), abs(electron.get_position()[2] - charge.get_position()[2])) #print distance
                break #exit this loop
        if abs(electron.get_position()[0]) > 100 or abs(electron.get_position()[1]) > 100 or abs(electron.get_position()[2]) > 100: #if it's outside of the boundaries of the simulation
            print("left boundaries")
            sep = False #don't continue
        if np.array_equal(electron.get_velocity(), [0.0,0.0,0.0]) and np.array_equal(electron.get_acceleration(), [0.0, 0.0,0.0]): #if the velocity + acceleration is 0
            print("no velocity/acceleration") #let us know that the velocity/acceleration is 0
            print(electron.get_velocity(), electron.get_acceleration()) #prove it
            sep = False #don't continue
        if sep == True: #if we still should continue
            i += 1 #up the frame number
            #print(i)
            yield i #iteration version of return

In [51]:
def create_animation(charges, electron): #creating the animation

    fig = plt.figure() #have a figure
    ax = fig.add_subplot(111, projection='3d')
    #print("h1")

    for c in charges:
        if c.get_charge() > 0:
            scatter = ax.scatter(c.get_position()[0], c.get_position()[1], c.get_position()[2], c="r", label="+ charges")
        else:
            scatter = ax.scatter(c.get_position()[0], c.get_position()[1], c.get_position()[2], c="b", label="- charges")
    scatter = ax.scatter(electron.get_position()[0], electron.get_position()[1], electron.get_position()[2], c="g", label="electron")
    #place the electron on the plot, coloring it green
    #print("h2")
    ax.legend() #show the legend
    #print("h3")
    def update(frame): #update function!

        electron_update(charges, electron) #call the update function to set the values
        print(electron.get_position())
        print(type(electron.get_position()))
        scatter._offsets3d = electron.get_position() #update the position of the electron
        print("hey")
        return scatter #return the new scatter plot

    electron.get_force(charges) #not have starting acceleration 0
    anim = animation.FuncAnimation(fig=fig, func=update, frames=gen(charges, electron), blit=False, interval=10, save_count=1000) #create the animation
    plt.close() #close the plot
    print("hi")
    return HTML(anim.to_jshtml()) #return the HTML to make the animation

In [53]:
c1 = Charge(1, 2, 3, e)
charges = [c1]
electron = Electron(-1, -2, -3)
create_animation(charges, electron)

hi
35
[ 4.66315176  9.32630352 13.98945529]
<class 'numpy.ndarray'>
hey
[ 8.63816388 17.27632777 25.91449165]
<class 'numpy.ndarray'>
hey


IndexError: invalid index to scalar variable.